# 영화 평점 분석 실습

In [49]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 1. 영화 평점 데이터 적재 및 전처리

In [50]:
# 사용자 데이터 읽어오기
users = pd.read_csv('data/movielens/users.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '성별','연령','직업','지역'])
users.head()

,사용자아이디,성별,연령,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [51]:
# 평점 데이터 읽어오기
ratings = pd.read_csv('data/movielens/ratings.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '영화아이디','평점','타임스탬프'])
ratings.head()

,사용자아이디,영화아이디,평점,타임스탬프
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [52]:
# 영화데이터 읽어오기
movies = pd.read_csv('data/movielens/movies.dat', sep = '::', engine = 'python',
                   names = ['영화아이디','영화제목','장르'], encoding = 'latin-1')
movies.head()

,영화아이디,영화제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [53]:
print('사용자수',len(users))
print('평점수',len(ratings))
print('영화수',len(movies))


사용자수 6040
평점수 1000209
영화수 3883


In [54]:
# 평점을 한번도 안 준 사용자가 있는가?  -> 없음
ratings.사용자아이디.nunique()

6040

In [55]:
# 평점이 하나도 없는 영화는 있나? -> 177개의 영화는 평점이 없음
ratings.영화아이디.nunique()

3706

In [56]:
#3개의 데이터프레임을 하나로 합치기
data = pd.merge(users, ratings)
data = pd.merge(data,movies)
data.head()

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


## 2. 보고 싶은 영화 찾기
영화들의 평점 평균을 구하여, 사람들에게 인정받는 (평점이 높은) 영화 찾기

In [57]:
# 영화제목이 중복되는게 있는지 체크
movies.nunique()

영화아이디    3883
영화제목     3883
장르        301
dtype: int64

In [58]:
# 영화들의 평점 평균을 구하여, 평점이 높은 영화 찾기 
data.pivot_table(index = ['영화아이디','영화제목'],
                values = '평점',
                aggfunc = 'mean').nlargest(10,'평점')


,,평점
영화아이디,영화제목,
787,"Gate of Heavenly Peace, The (1995)",5.0
989,Schlafes Bruder (Brother of Sleep) (1995),5.0
1830,Follow the Bitch (1998),5.0
3172,Ulysses (Ulisse) (1954),5.0
3233,Smashing Time (1967),5.0
3280,"Baby, The (1973)",5.0
3382,Song of Freedom (1936),5.0
3607,One Little Indian (1973),5.0
3656,Lured (1947),5.0


평균 평점이 만점인 영화들이 최상위에 위치함. 
일반적으로 평점이 만점인 경우는 대부분 평점의 개수가 매우 적은 경우이므로, 이를 확인하기 위해 평점의 개수도 함께 구해본다. 

In [59]:
# 영화들의 평점 평균을 구하여, 평점이 높은 영화 찾기 
data.pivot_table(index = ['영화아이디','영화제목'],
                values = '평점',
                aggfunc = ['mean','count']).nlargest(10,('mean','평점'))

,,mean,count
,,평점,평점
영화아이디,영화제목,,
787,"Gate of Heavenly Peace, The (1995)",5.0,3
989,Schlafes Bruder (Brother of Sleep) (1995),5.0,1
1830,Follow the Bitch (1998),5.0,1
3172,Ulysses (Ulisse) (1954),5.0,1
3233,Smashing Time (1967),5.0,2
3280,"Baby, The (1973)",5.0,1
3382,Song of Freedom (1936),5.0,1
3607,One Little Indian (1973),5.0,1


In [60]:
# 평점의 평균이 4.5이상이고, 평점의 개수도 1000개 이상인 영화를 보고 싶은 영화로 선정
ex = data.pivot_table(index = ['영화아이디','영화제목'],
                values = '평점',
                aggfunc = ['mean','count'])

ex.columns = ['평균','개수']

In [61]:
ex[(ex.평균 >= 4.5) & (ex.개수 >= 1000)]

,,평균,개수
영화아이디,영화제목,,
50,"Usual Suspects, The (1995)",4.517106,1783
318,"Shawshank Redemption, The (1994)",4.554558,2227
527,Schindler's List (1993),4.510417,2304
858,"Godfather, The (1972)",4.524966,2223


## [실습 #1] 여자들이 좋아하는 영화 찾기 
### - 여성 평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화

In [62]:
ex2 = data.pivot_table(index = '영화제목',
                columns = '성별',
                aggfunc = ['mean','count'],
                values = '평점')

In [63]:
여성인기영화 = ex2[(ex2[('mean','F')] >= 4.0) & (ex2[('count','F')] >= 500)]

## [실습 #2] 실습 #1에서 구한 영화(여성인기영화)의 장르를 분석해 보자.
여성인기영화의 장르 통계 구하기

예를 들어, 여성인기영화 중 Drama 장르의 영화는 10개, Action 영화는 3개, ...

In [64]:
# isin() 함수 활용
여성인기영화장르 = movies[movies.영화제목.isin(여성인기영화.index)].장르

In [65]:
여성인기영화장르 = 여성인기영화장르.str.split('|',expand=True)

In [66]:
여성인기영화장르

,0,1,2,3,4
0,Animation,Children's,Comedy,None,None
108,Action,Drama,War,None,None
257,Action,Adventure,Fantasy,Sci-Fi,None
293,Crime,Drama,None,None,None
315,Drama,None,None,None,None
352,Comedy,Romance,War,None,None
523,Drama,War,None,None,None
589,Drama,Thriller,None,None,None
604,Crime,Drama,Thriller,None,None
900,Drama,Romance,War,None,None


In [67]:
여성인기영화장르[0].value_counts().add(여성인기영화장르[1].value_counts(),fill_value = 0)

Action         7.0
Adventure      5.0
Animation      1.0
Children's     3.0
Comedy         4.0
Crime          3.0
Drama         10.0
Film-Noir      1.0
Romance        3.0
Sci-Fi         1.0
Thriller       2.0
War            1.0
dtype: float64

In [69]:
여성인기장르 = Series(dtype='float64')
for col in 여성인기영화장르.columns:
    여성인기장르 = 여성인기장르.add(여성인기영화장르[col].value_counts(), fill_value = 0)

In [71]:
여성인기장르.sort_values(ascending = False)

Drama         12.0
Action         7.0
Comedy         6.0
War            6.0
Adventure      5.0
Thriller       5.0
Romance        4.0
Sci-Fi         4.0
Children's     3.0
Crime          3.0
Fantasy        2.0
Animation      1.0
Film-Noir      1.0
Musical        1.0
Mystery        1.0
dtype: float64

## [실습 #3] 남자와 여자의 호불호가 크게 갈리는 영화 10개 찾기
전체 평점의 개수가 500개 이상인 영화만 대상으로 함.

In [81]:
ex3 = data.pivot_table(index = '영화제목', columns = '성별',
                      aggfunc =['mean','count'], values = '평점')

In [82]:
ex3

mean           count       
성별                                                 F         M     F      M
영화제목                                                                       
$1,000,000 Duck (1971)                      3.375000  2.761905  16.0   21.0
'Night Mother (1986)                        3.388889  3.352941  36.0   34.0
'Til There Was You (1997)                   2.675676  2.733333  37.0   15.0
'burbs, The (1989)                          2.793478  2.962085  92.0  211.0
...And Justice for All (1979)               3.828571  3.689024  35.0  164.0
...                                              ...       ...   ...    ...
Zed & Two Noughts, A (1985)                 3.500000  3.380952   8.0   21.0
Zero Effect (1998)                          3.864407  3.723140  59.0  242.0
Zero Kelvin (Kjærlighetens kjøtere) (1995)       NaN  3.500000   NaN    2.0
Zeus and Roxanne (1997)                     2.777778  2.357143   9.0   14.0
eXistenZ (1999)                             3.098592  3.289086  71.0  339.0

[3706 rows x 4 columns]

In [83]:
ex3_500 = ex3[ex3[('count','F')] + ex3[('count','M')] >= 500]

In [90]:
ex3_500.loc[:,'평점차이'] = \
    abs(ex3_500.loc[:,('mean','F')] - ex3_500.loc[:,('mean','M')])

In [94]:
ex3_500.sort_values(by = '평점차이', ascending = False).head(10)

mean            count         \
성별                                             F         M      F      M   
영화제목                                                                       
Dirty Dancing (1987)                    3.790378  2.959596  291.0  396.0   
Good, The Bad and The Ugly, The (1966)  3.494949  4.221300   99.0  723.0   
Dumb & Dumber (1994)                    2.697987  3.336595  149.0  511.0   
Evil Dead II (Dead By Dawn) (1987)      3.297297  3.909283   74.0  474.0   
Grease (1978)                           3.975265  3.367041  283.0  534.0   
Caddyshack (1980)                       3.396135  3.969737  207.0  760.0   
Animal House (1978)                     3.628906  4.167192  256.0  951.0   
Exorcist, The (1973)                    3.537634  4.067239  186.0  699.0   
Rocky Horror Picture Show, The (1975)   3.673016  3.160131  315.0  918.0   
Big Trouble in Little China (1986)      2.987952  3.485030   83.0  501.0   

                                            평점차이  
성별                                                
영화제목                                              
Dirty Dancing (1987)                    0.830782  
Good, The Bad and The Ugly, The (1966)  0.726351  
Dumb & Dumber (1994)                    0.638608  
Evil Dead II (Dead By Dawn) (1987)      0.611985  
Grease (1978)                           0.608224  
Caddyshack (1980)                       0.573602  
Animal House (1978)                     0.538286  
Exorcist, The (1973)                    0.529605  
Rocky Horror Picture Show, The (1975)   0.512885  
Big Trouble in Little China (1986)      0.497078

In [95]:
ex3_500.nlargest(10, '평점차이')

mean            count         \
성별                                             F         M      F      M   
영화제목                                                                       
Dirty Dancing (1987)                    3.790378  2.959596  291.0  396.0   
Good, The Bad and The Ugly, The (1966)  3.494949  4.221300   99.0  723.0   
Dumb & Dumber (1994)                    2.697987  3.336595  149.0  511.0   
Evil Dead II (Dead By Dawn) (1987)      3.297297  3.909283   74.0  474.0   
Grease (1978)                           3.975265  3.367041  283.0  534.0   
Caddyshack (1980)                       3.396135  3.969737  207.0  760.0   
Animal House (1978)                     3.628906  4.167192  256.0  951.0   
Exorcist, The (1973)                    3.537634  4.067239  186.0  699.0   
Rocky Horror Picture Show, The (1975)   3.673016  3.160131  315.0  918.0   
Big Trouble in Little China (1986)      2.987952  3.485030   83.0  501.0   

                                            평점차이  
성별                                                
영화제목                                              
Dirty Dancing (1987)                    0.830782  
Good, The Bad and The Ugly, The (1966)  0.726351  
Dumb & Dumber (1994)                    0.638608  
Evil Dead II (Dead By Dawn) (1987)      0.611985  
Grease (1978)                           0.608224  
Caddyshack (1980)                       0.573602  
Animal House (1978)                     0.538286  
Exorcist, The (1973)                    0.529605  
Rocky Horror Picture Show, The (1975)   0.512885  
Big Trouble in Little China (1986)      0.497078

## [실습 #4] 연령대 별로 영화 평점 분석하기
연령대(10대 미만, 10대, 20대, ...50대) 컬럼을 추가한 후, 영화별 연령대별 영화평점 구하기

In [96]:
data.describe()

,사용자아이디,연령,직업,영화아이디,평점,타임스탬프
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,2.973831e+01,8.036138e+00,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.175198e+01,6.531336e+00,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,2.500000e+01,2.000000e+00,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,2.500000e+01,7.000000e+00,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,3.500000e+01,1.400000e+01,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,5.600000e+01,2.000000e+01,3.952000e+03,5.000000e+00,1.046455e+09


In [97]:
# 1. 연령대 컬럼 추가 함수 사용
def generate_ages(x):
    if x < 10:
        return '10대 미만'
    elif x < 20:
        return '10대'
    elif x < 30:
        return '20대'
    elif x < 40:
        return '30대'
    elif x < 50:
        return '40대'
    else:
        return '50대 이상'

In [101]:
data['연령대'] = data.연령.apply(generate_ages)

In [104]:
data.head(5)

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르,연령대
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,10대 미만
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama,50대 이상
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama,20대
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama,20대
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama,50대 이상


In [105]:
# 두번째 방법 np.digitize()
import numpy as np
data['연령대2'] = np.digitize(data.연령,[10,20,30,40,50])

In [107]:
data.head(5)

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르,연령대,연령대2
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,10대 미만,0
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama,50대 이상,5
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama,20대,2
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama,20대,2
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama,50대 이상,5


In [108]:
data['연령대2'] = data.연령대2.map({
    0 : '10대 미만',
    1 : '10대',
    2 : '20대',
    3 : '30대',
    4 : '40대',
    5 : '50대 이상',
})

In [110]:
data.head(5)

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르,연령대,연령대2
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,10대 미만,10대 미만
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama,50대 이상,50대 이상
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama,20대,20대
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama,20대,20대
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama,50대 이상,50대 이상


In [111]:
# 그룹 집계
연령별평점 = data.pivot_table(index = '영화제목',
                        columns = '연령대', aggfunc = 'mean', values ='평점')

In [115]:
연령별평점 = 연령별평점[['10대 미만','10대','20대','30대','40대','50대 이상']]

In [116]:
연령별평점

연령대,10대 미만,10대,20대,30대,40대,50대 이상
영화제목,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.750000
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.600000
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.100000
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.674419
...,...,...,...,...,...,...
"Zed & Two Noughts, A (1985)",1.000000,3.000000,3.375000,3.777778,4.000000,3.000000
Zero Effect (1998),4.125000,3.883333,3.715278,3.608696,3.764706,3.769231
Zero Kelvin (Kjærlighetens kjøtere) (1995),NaN,NaN,NaN,3.500000,NaN,NaN
